<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/peft_merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install peft sentence-transformers bitsandbytes

In [2]:
from transformers import AutoTokenizer, MistralForCausalLM

model = MistralForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Hey, are you conscious? Can you talk to me?\n\nI’m not sure if you’re conscious or not. I’'

In [3]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

config = PeftConfig.from_pretrained("predibase/tldr_content_gen")
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, load_in_4bit=True, device_map="auto").eval()
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

model = PeftModel.from_pretrained(model, "predibase/tldr_content_gen", adapter_name="tldr_content_gen")
_ = model.load_adapter("predibase/tldr_headline_gen", adapter_name="tldr_headline_gen")

adapter_config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [6]:
adapters = ["tldr_content_gen", "tldr_headline_gen"]
weights = [1,1]
adapter_name = "merge"
density = 0.2
model.add_weighted_adapter(adapters, weights, adapter_name, combination_type="svd")

In [7]:
model.set_adapter("merge")

In [8]:
# Generate headline for an article
messages = [
    {"role": "user", "content":"""### Article: (CNN)Former Vice President Walter Mondale was released
     from the Mayo Clinic on Saturday after being admitted with influenza, hospital spokeswoman Kelley Luckstein said.
     "He's doing well. We treated him for flu and cold symptoms and he was released today," she said. Mondale, 87, was
     diagnosed after he went to the hospital for a routine checkup following a fever, former President Jimmy Carter said Friday.
     "He is in the bed right this moment, but looking forward to come back home," Carter said during a speech at a Nobel Peace
     Prize Forum in Minneapolis. "He said tell everybody he is doing well." Mondale underwent treatment at the Mayo Clinic in Rochester,
     Minnesota. The 42nd vice president served under Carter between 1977 and 1981, and later ran for President, but lost to Ronald Reagan.
     But not before he made history by naming a woman, U.S. Rep. Geraldine A. Ferraro of New York, as his running mate. Before that, the former
     lawyer was a U.S. senator from Minnesota. His wife, Joan Mondale, died last year.###
     Content:"""
    },
]


device = 'cuda'
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt", )

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=256, do_sample=True, top_p=0.9, temperature=0.5, repetition_penalty=1.2, eos_token_id=2, pad_token_id=2)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


<s> [INST] ### Article: (CNN)Former Vice President Walter Mondale was released
     from the Mayo Clinic on Saturday after being admitted with influenza, hospital spokeswoman Kelley Luckstein said.
     "He's doing well. We treated him for flu and cold symptoms and he was released today," she said. Mondale, 87, was 
     diagnosed after he went to the hospital for a routine checkup following a fever, former President Jimmy Carter said Friday.
     "He is in the bed right this moment, but looking forward to come back home," Carter said during a speech at a Nobel Peace 
     Prize Forum in Minneapolis. "He said tell everybody he is doing well." Mondale underwent treatment at the Mayo Clinic in Rochester,
     Minnesota. The 42nd vice president served under Carter between 1977 and 1981, and later ran for President, but lost to Ronald Reagan. 
     But not before he made history by naming a woman, U.S. Rep. Geraldine A. Ferraro of New York, as his running mate. Before that, the former
    

In [16]:
# Generate article for headline
messages = [
    {"role": "user", "content":"""The following headline is the headline of a news report.
Please write the content of the news passage based on only this headline.\n\n
Headline: Latest success from Google’s AI group: Controlling a fusion reactor \n\nContent:"""
    },
]

device = 'cuda'
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt", )

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True, top_p=0.9, temperature=0.9, repetition_penalty=1.2, eos_token_id=1, pad_token_id=2)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> [INST] The following headline is the headline of a news report. 
Please write the content of the news passage based on only this headline.


Headline: Latest success from Google’s AI group: Controlling a fusion reactor 

Content: [/INST]</s>вьи (integrierte Verbrennungskraftwerke) means integrated nuclear power plants in Germany, where the fusion-powered ITER project has been running for over a decade now. This new article explores whether AI can be used to improve the process and make it more efficient? Google's latest research focuses on using DeepMind technology to control a fusion reactor so that all its components are performing at their best possible efficiency. It would seem that the most successful use cases for AI have been those where computers need to analyze large amounts of data quickly. However, it could be argued that many machine learning applications aren't really about AI at all.</s>вьи is an interesting read with lots of technical detail that is easy to follow al

In [23]:
# Try to generate both
messages = [
    {"role": "user", "content":"""Provide an english headline and article content for car sales in the format \n ###Format: Headline - your headline \n Content - your content \n\nContent:"""},
]

device = 'cuda'
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt", )

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=124, do_sample=True, top_p=0.9, temperature=0.9, repetition_penalty=1.2, eos_token_id=1, pad_token_id=2)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> [INST] Provide an english headline and article content for car sales in the format 
 ###Format: Headline - your headline 
 Content - your content 

Content: [/INST]</s>рные автомобили по всей России и ближние страны. Заказать машину можно на сайте или в одном из офисов компании! (Motorsports Cars All Over Russia & Nearby Countries. You can order a car at our site or any of our offices!)</s>рные автомобили по всей России...</s>рный автомобиль (Russian Sports Car)</s>рные автомобили по всей России и ближние страна ...</s>рный автомобиль (Sporty Russian
